# Generate Call data names...

Building on the work I did in bbgen2.ipynb, this time around I'm going to construct a table to map from the call data
numeric value to a string name, which I suspect will be beneficial for those trying to interpret the data.



In [1]:
import os
import sys

In [1]:
fuse_lowlevel_h = "../../include/fuse_lowlevel.h"

with open(fuse_lowlevel_h, "rt") as fd:
    fuse_ll_lines = fd.readlines()

index = 0
functions = {}
for l in fuse_ll_lines:
    # Step (1): find the start of the operations definition
    index = index + 1
    if 'struct fuse_lowlevel_ops' in l: break
while index < len(fuse_ll_lines):
    if 'void (*' in fuse_ll_lines[index]:
        func = fuse_ll_lines[index][2+fuse_ll_lines[index].find('('):fuse_ll_lines[index].find(')')]
        functions[func] = fuse_ll_lines[index].rstrip()
        while True:
            if ');' in fuse_ll_lines[index]: break
            index = index + 1
            functions[func] += fuse_ll_lines[index].rstrip()
    if fuse_ll_lines[index].startswith('};'): break
    index = index + 1
titles = [x.title() for x in functions]

preamble = '''
//
// (C) Copyright 2020 Tony Mason
// All Right Reserved
//
//

#include "bitbucketcalls.h"


char * const * BitbucketCallDataNames[] = {
'''

with open('calldata.c', 'wt') as fd:
    fd.write(preamble)
    for t in titles:
        fd.write('\t"' + t + '",\n')
    fd.write('};\n\n')

In [ ]:
symbols = []
calls = []
for func in functions:
    cfunc = functions[func]
    proto_start = cfunc.find('(*')
    proto_end = cfunc.find(')(')
    proto_end_length = 1
    if proto_end <= proto_start:
        proto_end = cfunc.find(') (')
        proto_end_length = 2
    assert proto_end > proto_start, '{}: start = {}, end = {}'.format(func, proto_start, proto_end)
    statement_end = cfunc.find(';')
    sym = 'BITBUCKET_CALL_' + cfunc[proto_start + 2: proto_end]
    symbols.append(sym.upper())
    calls.append(cfunc[proto_start + 2: proto_end])
    
for sym in symbols:
    index = index + 1
    sym = '#define ' + sym + ' (BITBUCKET_CALL_BASE + {})'.format(index)
    defs.append(sym)